In [46]:
import pysam
import logging
import sys
import pandas as pd
from Bio import SeqIO

In [47]:
#################################################################################
# FUN
#################################################################################
def load_reference_fasta_as_dict(ref_fasta_path, ref_name_list="All", log_verbose=30):
    """
    INPUT:
        <ref_fasta_path>
            Reference fasta file path
        
        <ref_name_list>
            If set All, load all seq info in reference, else only try to load seq_id in the list
    
    RETURN
        <ref_seq_dict>
            A dict, key is seq_id and value is sequence with  .upper()
        
        None
            If occur error, return None.
    """

    # ---------------------------------------------------------->>>>>>>>>>>>>>>>>>>>
    # log setting
    # ---------------------------------------------------------->>>>>>>>>>>>>>>>>>>>
    logging.basicConfig(level=(4 - log_verbose) * 10,
                        format='%(levelname)-5s @ %(asctime)s: %(message)s ',
                        datefmt='%Y-%m-%d %H:%M:%S',
                        stream=sys.stderr,
                        filemode="w")

    # ---------------------------------------------------------->>>>>>>>>>>>>>>>>>>>
    # load genome as dict
    # ---------------------------------------------------------->>>>>>>>>>>>>>>>>>>>
    try:
        genome_fa = SeqIO.parse(handle=ref_fasta_path, format="fasta")
    except:
        raise IOError("Load file error! %s" % ref_fasta_path)

    # init var
    ref_seq_dict = {}
    ref_name_set = set(ref_name_list)

    logging.info("Starting to load the reference genome...")

    for ref in genome_fa:
        if ref_name_list == "All":
            ref_seq_dict[ref.id] = ref.seq.upper()
            logging.debug("Loading genome...\t" + ref.id)

        elif ref.id in ref_name_list:
            ref_seq_dict[ref.id] = ref.seq.upper()
            logging.debug("Loading genome...\t" + ref.id)

            # remove already loaded seq
            ref_name_set.remove(ref.id)

            # load all info
            if len(ref_name_set) == 0:
                break

    logging.info("Loading genome done!")

    return ref_seq_dict


def get_align_mismatch_pairs(align, ref_genome_dict=None):
    """
    INPUT
        <align>
            pysam AlignedSegment object

    RETURN
        <mismatch_pair_list>
            [ref_index, align_index, ref_base, align_base]

            ref_index is the same coordinate with UCSC genome browser

            When NM == 0, return None
    """
    # No mismatch
    try:
        if align.get_tag("NM") == 0:
            return None
    except:
        return None
    
    MD_tag_state = align.has_tag("MD")

    if MD_tag_state:
        # parse softclip, insertion and deletion
        info_index_list = []
        accu_index = 0

        for cigar_type, cigar_len in align.cigartuples:
            if cigar_type == 1 or cigar_type == 4:
                info_index_list.append((accu_index + 1, cigar_len))

            elif cigar_type == 2:
                info_index_list.append((accu_index + 1, -cigar_len))

            accu_index += cigar_len

        # parse MD tag
        mismatch_pair_list = []
        cur_base = ""
        cur_index = 0
        bases = align.get_tag("MD")

        i = 0
        while i < len(bases):
            base = bases[i]

            if base.isdigit():
                cur_base += base
                i += 1

            else:
                cur_index += int(cur_base)
                cur_base = ""

                if base == "^":
                    i += 1
                    del_str = ""

                    while (bases[i].isalpha()) and (i < len(bases)):
                        del_str += bases[i]
                        i += 1

                    cur_index += len(del_str)
                    del_str = ""

                elif base.isalpha():
                    cur_index += 1
                    ref_base = base
                    i += 1

                    # add into list
                    fix_index = cur_index + back_indel_shift(info_index_list, cur_index)

                    if fix_index < len(align.query_sequence):
                        mismatch_pair_list.append([cur_index + align.reference_start, cur_index - 1, ref_base,
                                                   align.query_sequence[fix_index - 1]])
                    else:
                        return None

        return mismatch_pair_list
    else:
        mismatch_pair_list = []
        for align_idx, ref_idx in align.get_aligned_pairs():
            if (align_idx is not None) and (ref_idx is not None):
                align_base = align.query_sequence[align_idx]
                ref_base = ref_genome_dict[align.reference_name][ref_idx]

                if align_base != ref_base:
                    mismatch_pair_list.append([
                        ref_idx + 1,
                        align_idx,
                        ref_base,
                        align_base
                    ])

        return mismatch_pair_list
    
def back_indel_shift(info_index_list, cur_index):
    """
    INPUT:
        <info_index_list>
            generated from align.cigar tuples

        <cur_index>
            index related to MD tag in BAM file

    RETURN
        <acc_shift>
    """

    # parse soft clip and insertion
    if len(info_index_list) == 0:
        return 0

    acc_shift = 0
    for info_start_index, info_len in info_index_list:
        if info_start_index >= cur_index:
            return acc_shift

        else:
            acc_shift += info_len

    return acc_shift


def aligned_segment_get_reference_sequence(align,ref_genome_dict):
    """
    INPUT
        <align>
            pysam AlignedSegment object

        <ref_genome_dict>
            dict generated by func <load_reference_fasta_as_dict>

    RETURN
        <Bio Seq object>
    """
    MD_tag_state = align.has_tag("MD")

    align_start_idx = align.get_aligned_pairs()[0][0]
    ref_start_idx = align.get_aligned_pairs()[0][1]
    align_end_idx = align.get_aligned_pairs()[-1][0]
    ref_end_idx = align.get_aligned_pairs()[-1][1]

    if MD_tag_state:
        return align.get_reference_sequence().__str__()
    else:
        return ref_genome_dict[align.reference_name][ref_start_idx:ref_end_idx+1]

In [48]:
ref_fasta_path = "/gpfs/user/zhaohuanan/2.database/fasta_hg38/hg38_only_chromosome.fa"
vcf = "/gpfs/user/zhaohuanan/2.database/cell_line_mutations/293T/293T_BE_INPUT_VCF/293T-Mock-Input-covaris_bwa_hg38_sort_rmdup.recall.merge.Genotype.filter.rmdup_signal.vcf"
mut_bed = "/gpfs/user/zhaohuanan/2.database/cell_line_mutations/293T/293T_BE_INPUT_VCF/293T-EMX1-Mock-Input.site_index.rmdup.bed"

num_extend = 100
has_bed_header = True
ls_mut_direction = ['CT','GA']


with open(vcf, 'r') as vcf:
    ls_vcf = [i.strip().split('\t')[:2] + i.strip().split('\t')[3:5] for i in vcf.readlines() if i[0]!='#']
ls_filterd_vcf = [i for i in ls_vcf if (i[2]=='C' and i[3]=='T') or (i[2]=='G' and i[3]=='A')]

with open(mut_bed,'r') as bed:
    ls_bed = [i.strip().split('\t')[3].split('_') for i in bed.readlines()]
    ls_bed = [i[:2] + [j for j in i[2]] for i in ls_bed]

ls_endogenous_mut = ls_filterd_vcf + ls_bed
ls_endogenous_mut.sort()
ls_endogenous_mut[:10]

[['chr1', '1000018', 'G', 'A'],
 ['chr1', '10000789', 'G', 'A'],
 ['chr1', '10002030', 'G', 'A'],
 ['chr1', '100056713', 'G', 'A'],
 ['chr1', '100056713', 'G', 'A'],
 ['chr1', '100089610', 'C', 'T'],
 ['chr1', '100089610', 'C', 'T'],
 ['chr1', '100093453', 'G', 'A'],
 ['chr1', '100093453', 'G', 'A'],
 ['chr1', '100100720', 'C', 'T']]

In [49]:
dt_endogenous_mut = {}
for record in ls_endogenous_mut:
    dt_endogenous_mut["_".join(record[:3])+record[3]] = record
# dt_endogenous_mut

In [50]:
ref_dict = load_reference_fasta_as_dict(ref_fasta_path=ref_fasta_path)

INFO  @ 2021-01-12 13:49:30: Starting to load the reference genome... 
DEBUG @ 2021-01-12 13:49:33: Loading genome...	chr1 
DEBUG @ 2021-01-12 13:49:36: Loading genome...	chr2 
DEBUG @ 2021-01-12 13:49:38: Loading genome...	chr3 
DEBUG @ 2021-01-12 13:49:40: Loading genome...	chr4 
DEBUG @ 2021-01-12 13:49:42: Loading genome...	chr5 
DEBUG @ 2021-01-12 13:49:44: Loading genome...	chr6 
DEBUG @ 2021-01-12 13:49:45: Loading genome...	chr7 
DEBUG @ 2021-01-12 13:49:47: Loading genome...	chr8 
DEBUG @ 2021-01-12 13:49:48: Loading genome...	chr9 
DEBUG @ 2021-01-12 13:49:50: Loading genome...	chr10 
DEBUG @ 2021-01-12 13:49:51: Loading genome...	chr11 
DEBUG @ 2021-01-12 13:49:53: Loading genome...	chr12 
DEBUG @ 2021-01-12 13:49:54: Loading genome...	chr13 
DEBUG @ 2021-01-12 13:49:55: Loading genome...	chr14 
DEBUG @ 2021-01-12 13:49:56: Loading genome...	chr15 
DEBUG @ 2021-01-12 13:49:57: Loading genome...	chr16 
DEBUG @ 2021-01-12 13:49:58: Loading genome...	chr17 
DEBUG @ 2021-01-12 1

In [51]:
# 1
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/293T-bat_EMX1-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20201229-EMX1-off-target_regions.IgvChecked.AddDetectSeqV2Index.bed'
# 2
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/293T-bat_EMX1-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20201229-EMX1-off-target_regions.IgvChecked.AddDetectSeqV2Index.bed'
# 3
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/293T-bat_HN-HEK4-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20210108-293T-HEK4-off-target_regions.IgvChecked.AddDetectSeqV2Index.AddWeak.bed'
# 4
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/293T-bat_HN-HEK4-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20210108-293T-HEK4-off-target_regions.IgvChecked.AddDetectSeqV2Index.AddWeak.bed'
# 5
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/MCF7-bat_HR-HEK4-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20210104-MCF7-HEK4-off-target_regions.IgvChecked.AddDetectSeqV2Index.bed'
# 6
path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/MCF7-bat_HR-HEK4-All-PD_rep2_hg38.MAPQ20.bam"
bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20210104-MCF7-HEK4-off-target_regions.IgvChecked.AddDetectSeqV2Index.bed'
# 7
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/MCF7-bat_HR-RNF2-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20201210-NCF7-RNF2-Detect-seq_on-target.bed'
# 8
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/MCF7-bat_HR-RNF2-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20201210-NCF7-RNF2-Detect-seq_on-target.bed'

# 9
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/293T-bat_VEGFA-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20201120-VEGFA-off-target_regions.IgvChecked.AddDetectSeqV2Index.bed'

# 10
# path_bam = "/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bam/293T-bat_VEGFA-All-PD_rep2_hg38.MAPQ20.bam"
# bed_file = '/gpfs/user/zhaohuanan/3.project/25.2020-12_02_CBE_DetectSeq_paper_revise/bed_final_list/20201120-VEGFA-off-target_regions.IgvChecked.AddDetectSeqV2Index.bed'

# test
# path_bam = "/Users/mac/mac_data2/FilesForTest/bam/293T-bat_VEGFA-All-PD_rep1_hg38.MAPQ20.bam"
# bed_file = '20200611-293T-VEGFA-Detect-seq_pRBS.bed'

In [52]:
bam_file = pysam.AlignmentFile(path_bam, "rb")


with open(bed_file, "r") as f:
    ls_bed_info = [i.strip().split('\t') for i in f.readlines()]

if has_bed_header:
    ls_bed_info = ls_bed_info[1:]
ls_bed_info[:3]

[['chr1', '10692448', '10692470', 'MCF7-HEK4-Detect-off-target-32', '5', '+'],
 ['chr1',
  '18880987',
  '18881009',
  'MCF7-HEK239T-site4-Detect-off-target-10',
  '3',
  '-'],
 ['chr1',
  '32006042',
  '32006064',
  'MCF7-HEK239T-site4-Detect-off-target-19',
  '4',
  '+']]

In [53]:
ls_chr = ['chr%s' % str(i+1) for i in range(22)] + ['chr%s' % i for i in ["X","Y","M"]]
# ls_chr

In [54]:
ls_table = []
for ls_bed_line in ls_bed_info:
    chr_ = ls_bed_line[0]
    start =int(ls_bed_line[1]) - num_extend
    end = int(ls_bed_line[2]) + num_extend
    region_index = ls_bed_line[3].strip()
    strand = ls_bed_line[5]
    
    
    for index, align in enumerate(bam_file.fetch(contig=chr_, start=start, end=end)):
        # todo debug
#         if align.query_name == "E00528:558:H73NTCCX2:3:1120:9800:70574":
#             print(align, "\n%s\t%s\t%s" % (chr_, start, end))
#         else:
#             continue
#         print(index, '\n', align.query_name, '\n', align.get_aligned_pairs())
        MD_tag_state = align.has_tag("MD")

#         if MD_tag_state:
#             mis_align_pair_list = get_align_mismatch_pairs(align)
#         else:
        mis_align_pair_list = get_align_mismatch_pairs(align=align, ref_genome_dict=ref_dict)
#         print(mis_align_pair_list)
        # 0region_index, 1reads_id, 2align_strand, 3extend_length,4CT_count, 5Ct_max_gap, 6GA_count, 7GA_max_gap, 8CT_event_Cs_passing_by_at_fwd_strand, 9G2A_event_Gs_passing_by_at_rev_strand
        
        # 如果这个reads至少存在一个mutation信息
        if mis_align_pair_list and align.is_paired and not align.is_unmapped:
            # 获取chr信息，从数字转为chrX这种
            chr_name = ls_chr[align.rname]
            
            # 去除mis align pair list中的固有mutations
            check_mis_align_pair_list = []
            for ls_mut in mis_align_pair_list:
                mut_key = "%s_%s_%s%s" % (chr_name, ls_mut[0], ls_mut[2], ls_mut[3])

                if not dt_endogenous_mut.__contains__(mut_key):
                    check_mis_align_pair_list.append(ls_mut)

            mis_align_pair_list = check_mis_align_pair_list
            # todo debug
#             print(mis_align_pair_list)
            #----
            # 如果mis_align_pair_list为空，则进入下一个循环
            if mis_align_pair_list == []:
                continue
            # checked strand:
            if align.is_reverse:
                align_strand = "rev"
            else:
                align_strand = 'fwd'
            
            # add pe-reads info
            if align.is_read1:
                pe_reads = "R1"
            else:
                pe_reads = 'R2'

            aligned_reads_id = align.query_name
            extend_length = num_extend

            # 如果只存在一个mutation，不做判断，直接存入list
            if len(mis_align_pair_list)==1:
                ls_line = []
                if mis_align_pair_list[0][2]=='C' and mis_align_pair_list[0][3]=='T' and align_strand == 'fwd':
                    ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 1, 0, 0, 0, 1, 0]
                elif mis_align_pair_list[0][2]=='C' and mis_align_pair_list[0][3]=='T' and align_strand == 'rev':
                    ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 0, 0, 1, 0, 0, 1]
                elif mis_align_pair_list[0][2]=='G' and mis_align_pair_list[0][3]=='A' and align_strand == 'fwd':
                    ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 0, 0, 1, 0, 0, 1]
                elif mis_align_pair_list[0][2]=='G' and mis_align_pair_list[0][3]=='A' and align_strand == 'rev':
                    ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 1, 0, 0, 0, 1, 0]
                else:
                    pass

            # 如果存在两个或两个以上的mutation        
            else:
                ls_line1 = []
                ls_line2 = []
                ls_line = []
                
                # 核心！！！！！！！！！
                # 先把CT的全筛出来
                ls_this_reads_info = [site for site in mis_align_pair_list if site[2]=='C' and site[3]=='T']
                # 如果筛不到，没有就没有
                # 如果筛到至少一个
                if ls_this_reads_info:
                    # 如果有一个就加到ls_table中
                    if len(ls_this_reads_info)==1:
                        if align_strand == 'fwd':
                            ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 1, 0, 0, 0, 1, 0]
                        else:
                            ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 0, 0, 1, 0, 0, 1]
                    # 如果有更多再计算count和gap
                    else:# todo 
                        # the first C info and the last C info from ls_this_reads_info from mis_align_pair_list
                        ls_first = ls_this_reads_info[0]
                        ls_last = ls_this_reads_info[-1]
                        # count Cs passing by
                        c_passing_by = aligned_segment_get_reference_sequence(align=align,ref_genome_dict=ref_dict)[ls_first[1]:ls_last[1]+1].upper().count('C')
#                         print(c_passing_by)
                        if align_strand == 'fwd':
                            ls_line1 = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, len(ls_this_reads_info),ls_last[0]-ls_first[0], 0, 0, c_passing_by, 0]
                        else:
                            ls_line1 = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 0, 0, len(ls_this_reads_info),ls_last[0]-ls_first[0], 0, c_passing_by]
                            
                # 核心！！！！！！！！！
                # 再把GA的全筛出来
                ls_this_reads_info = [site for site in mis_align_pair_list if site[2]=='G' and site[3]=='A']
                # 如果筛不到，没有就没有
                # 如果筛到至少一个
                if ls_this_reads_info:
                    # 如果有一个就加到ls_table中
                    if len(ls_this_reads_info)==1:
                        if align_strand == 'fwd':
                            ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 0, 0, 1, 0, 0, 1]
                        else:
                            ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 1, 0, 0, 0, 1, 0]
                    # 如果有更多再计算count和gap
                    else:# todo 
                        ls_first = ls_this_reads_info[0]
                        ls_last = ls_this_reads_info[-1]
                        g_passing_by = aligned_segment_get_reference_sequence(align=align,ref_genome_dict=ref_dict)[ls_first[1]:ls_last[1]+1].upper().count('G') # 这里要count 的那个strand上的G，就是原来链上的C
#                         print(align.get_reference_sequence()[ls_first[1]:ls_last[1]+1].upper())
#                         print(c_passing_by)
                        if align_strand == 'fwd':
                            ls_line2 = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, 0, 0, len(ls_this_reads_info),ls_last[0]-ls_first[0], 0, g_passing_by]
                        else:
                            ls_line2 = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length, len(ls_this_reads_info),ls_last[0]-ls_first[0], 0, 0, g_passing_by, 0]
                            
                if ls_line1 and ls_line2:
                    ls_line = [region_index, strand, aligned_reads_id, pe_reads, align_strand, extend_length,
                               ls_line1[6]+ls_line2[6], ls_line1[7]+ls_line2[7], ls_line1[8]+ls_line2[8], 
                               ls_line1[9]+ls_line2[9], ls_line1[10]+ls_line2[10], ls_line1[11]+ls_line2[11],]
                elif ls_line1:
                    ls_line = ls_line1
                elif ls_line2:
                    ls_line = ls_line2
                else:
                    pass
                if ls_line:
                    ls_table.append(ls_line)
                else:
                    pass
                    
                # 既没有CT也没有GA的直接pass

In [55]:
df = pd.DataFrame(ls_table, columns=["region_index", "strand", "reads_id","pe_index" , "align_orientation", 
                                     "bed_region_extend_length_each_term", "count_C2T", "max_distance_C2T", "count_G2A", "max_distance_G2A", 
                                     "CT_event_Cs_passing_by", "GA_event_Gs_passing_by"])
df

,region_index,strand,reads_id,pe_index,align_orientation,bed_region_extend_length_each_term,count_C2T,max_distance_C2T,count_G2A,max_distance_G2A,CT_event_Cs_passing_by,GA_event_Gs_passing_by
0,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C002R0480428280/1,R1,fwd,100,2,1,0,0,2,0
1,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C002R0030369199/1,R1,fwd,100,3,3,0,0,3,0
2,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C001R0101248977,R1,fwd,100,3,3,0,0,3,0
3,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C002R0220197236/1,R1,fwd,100,3,4,0,0,3,0
4,MCF7-HEK4-Detect-off-target-32,+,F300001684AL1C003R0340680630/1,R1,fwd,100,5,59,0,0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3562,MCF7-HEK4-Detect-off-target-129,-,F300001684AL2C002R0451215449/1,R1,rev,100,1,0,0,0,1,0
3563,MCF7-HEK4-Detect-off-target-129,-,F300001684AL2C002R0440244124/1,R2,fwd,100,0,0,1,0,0,1
3564,MCF7-HEK4-Detect-off-target-129,-,F300001684AL1C003R0330199012/1,R1,rev,100,1,0,0,0,1,0
3565,MCF7-HEK4-Detect-off-target-129,-,F300001684AL2C004R0010127472/1,R1,rev,100,2,5,0,0,2,0


In [56]:
df = df.drop_duplicates(subset=['reads_id', 'pe_index'],keep=False)
df

,region_index,strand,reads_id,pe_index,align_orientation,bed_region_extend_length_each_term,count_C2T,max_distance_C2T,count_G2A,max_distance_G2A,CT_event_Cs_passing_by,GA_event_Gs_passing_by
0,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C002R0480428280/1,R1,fwd,100,2,1,0,0,2,0
1,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C002R0030369199/1,R1,fwd,100,3,3,0,0,3,0
2,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C001R0101248977,R1,fwd,100,3,3,0,0,3,0
3,MCF7-HEK4-Detect-off-target-32,+,F300001684AL2C002R0220197236/1,R1,fwd,100,3,4,0,0,3,0
4,MCF7-HEK4-Detect-off-target-32,+,F300001684AL1C003R0340680630/1,R1,fwd,100,5,59,0,0,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3562,MCF7-HEK4-Detect-off-target-129,-,F300001684AL2C002R0451215449/1,R1,rev,100,1,0,0,0,1,0
3563,MCF7-HEK4-Detect-off-target-129,-,F300001684AL2C002R0440244124/1,R2,fwd,100,0,0,1,0,0,1
3564,MCF7-HEK4-Detect-off-target-129,-,F300001684AL1C003R0330199012/1,R1,rev,100,1,0,0,0,1,0
3565,MCF7-HEK4-Detect-off-target-129,-,F300001684AL2C004R0010127472/1,R1,rev,100,2,5,0,0,2,0


In [57]:
tsv_path = path_bam.replace("/bam/", "/MutationRegionCount.table/").replace("_hg38.MAPQ20.bam", "_max_distance_count.tsv")
try:
    os.makedirs("/".join(tsv_path.split('/')[:-1]))
except:
    pass

In [58]:
df.to_csv(tsv_path, sep="\t")
# df.to_csv("./test.tsv", sep="\t")
bam_file.close()